### **Model**

In [2]:
import openai
from sentence_transformers import SentenceTransformer
from sklearn.cluster import KMeans
import numpy as np
from fuzzywuzzy import fuzz

openai.api_key = 'your_openai_api_key'

def load_model(multi=True):
    if multi:
        model = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2')
    else:
        model = SentenceTransformer('all-MiniLM-L6-v2')
    return model

def semantic_clustering(corpus, model, k=10):
    corpus_embeddings = model.encode(corpus)
    
    kmeans = KMeans(n_clusters=k, random_state=0)
    clusters = kmeans.fit_predict(corpus_embeddings)

    cluster_groups = {i: [] for i in range(k)}
    for idx, cluster_id in enumerate(clusters):
        cluster_groups[cluster_id].append(corpus[idx])
    
    labels = assign_labels(cluster_groups)
    
    return clusters, labels

def assign_labels(cluster_groups):
    labels = []
    for cluster_id, items in cluster_groups.items():
        prompt = f"Assign a semantic label to the following claims:\n"
        for item in items:
            prompt += f"- {item}\n"
        response = openai.Completion.create(
            model="gpt-4",
            prompt=prompt,
            max_tokens=10
        )
        label = response['choices'][0]['text'].strip()
        labels.append(label)
    return labels

def fuzzymatch(query, labels, cluster_groups):
    matched_clusters = []
    for idx, label in enumerate(labels):
        score = fuzz.ratio(query, label)
        if score > 60:  # Adjust threshold
            matched_clusters.append(cluster_groups[idx])
    return matched_clusters

def evaluate_supportiveness(query, cluster, model='gpt-4'):
    prompt = f"Evaluate if the following claims support this query: {query}\n"
    for claim in cluster:
        prompt += f"- {claim}\n"
    prompt += "Mark as 'supportive' or 'not supportive' for each claim."
    
    response = openai.Completion.create(
        model=model,
        prompt=prompt,
        max_tokens=50
    )
    
    result = response['choices'][0]['text'].strip().split('\n')
    supportiveness_scores = [1 if 'supportive' in r.lower() else 0 for r in result]
    
    supportive_claims = [cluster[i] for i, score in enumerate(supportiveness_scores) if score == 1]
    return supportive_claims

# FASTTRACK Algorithm implementation
def fasttrack_algorithm(queries, corpus, multi=True):
    model = load_model(multi)
    Dsel = []
    
    # Stage 1: Semantic Clustering
    clusters, labels = semantic_clustering(corpus, model, k=10)
    
    # Stage 2: Tracing (for each query)
    for query in queries:
        Dq = [] 
        
        matched_clusters = fuzzymatch(query, labels, clusters)
        
        for cluster in matched_clusters:
            supportive_claims = evaluate_supportiveness(query, cluster)
            Dq.extend(supportive_claims)
        
        Dsel.extend(Dq)
    
    return Dsel
